# Testing on Adventureworks DB

## Update statements on 30 threads

In [5]:
!uv pip install psycopg2-binary
!uv pip install matplotlib
!uv pip install plotly
!uv pip install pandas
!uv pip install nbformat

Audited 1 package in 0.47ms
Audited 1 package in 1ms
Audited 1 package in 0.60ms
Audited 1 package in 0.88ms
Resolved 10 packages in 551ms                                                
Installed 7 packages in 8ms10.4                                     
 + attrs==23.2.0
 + fastjsonschema==2.20.0
 + jsonschema==4.22.0
 + jsonschema-specifications==2023.12.1
 + nbformat==5.10.4
 + referencing==0.35.1
 + rpds-py==0.18.1


In [1]:
import sys
import time
import psycopg2
import psycopg2.extensions
from psycopg2.extras import LoggingConnection, LoggingCursor
from random import randint 
import logging
from concurrent.futures import ThreadPoolExecutor

In [2]:
logging.basicConfig(
     filename='nonimv.log',
    #  stream=sys.stdout,
     level=logging.DEBUG,
     filemode='w',
     format="%(message)s"
 )
logger = logging.getLogger(__name__)

# MyLoggingCursor simply sets self.timestamp at start of each query                                                                 
class MyLoggingCursor(LoggingCursor):
    def execute(self, query, vars=None):
        self.timestamp = time.time()
        return super(MyLoggingCursor, self).execute(query, vars)

    def callproc(self, procname, vars=None):
        self.timestamp = time.time()
        return super(MyLoggingCursor, self).callproc(procname, vars)

# MyLogging Connection:                                                                                                             
#   a) calls MyLoggingCursor rather than the default                                                                                
#   b) adds resulting execution (+ transport) time via filter()                                                                     
class MyLoggingConnection(LoggingConnection):
    def filter(self, msg, curs):
        # return msg + "   %d ms" % int((time.time() - curs.timestamp) * 1000)
        return int((time.time() - curs.timestamp) * 1000)

    def cursor(self, *args, **kwargs):
        kwargs.setdefault('cursor_factory', MyLoggingCursor)
        return LoggingConnection.cursor(self, *args, **kwargs)

def update_nonimv_worker(id):
    db_settings = {"dbname":"Adventureworks", "host":"localhost","port":5432,"user":"postgres","password":"123qwe"}
    conn = psycopg2.connect(connection_factory=MyLoggingConnection, **db_settings)
    conn.initialize(logger)

    table_name = "sales.salesorderdetailnonimv"

    query_txt = f"""UPDATE {table_name}
    SET orderqty={randint(1,100)}
    WHERE salesorderdetailid = {id+1}"""

    cur = conn.cursor()
    cur.execute(query_txt)
    conn.commit()
    cur.close()
    conn.close()

def update_imv_worker(id):
    db_settings = {"dbname":"Adventureworks", "host":"localhost","port":5432,"user":"postgres","password":"123qwe"}
    conn = psycopg2.connect(connection_factory=MyLoggingConnection, **db_settings)
    conn.initialize(logger)

    table_name = "sales.salesorderdetailimv"

    query_txt = f"""UPDATE {table_name}
    SET orderqty={randint(1,100)}
    WHERE salesorderdetailid = {id+1}"""

    cur = conn.cursor()
    cur.execute(query_txt)
    conn.commit()
    cur.close()
    conn.close()

In [4]:
with ThreadPoolExecutor(30) as pool:
    # pool.map(update_worker,range(1000))
    jobs = [pool.submit(update_nonimv_worker , x) for x in range(3000)]


In [5]:
logging.basicConfig(
     filename='imv.log',
    #  stream=sys.stdout,
     level=logging.DEBUG,
     filemode='w',
     format="%(message)s",
     force=True
 )

In [7]:
with ThreadPoolExecutor(30) as pool:
    # pool.map(update_worker,range(1000))
    jobs = [pool.submit(update_imv_worker , x) for x in range(3000)]


# Calculations

In [15]:
with open("nonimv.log", "r") as f:
    _l =f.read()
    l = _l.split("\n")
    l = list(map(int,l))
with open("imv.log", "r") as f:
    _i =f.read()
    i = _i.split("\n")
    i = list(map(int,i))

In [31]:
import numpy as np

print("avg diff: ",  np.average(i) - np.average(l), " ms")
print("avg fold: ",  np.average(i)/np.average(l), " x")

avg diff:  84.95466666666667  ms
avg fold:  5.340105239854912  x


In [32]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Violin(
                        y=l,
                        name="original",
                        box_visible=True,
                        meanline_visible=True))

fig.add_trace(go.Violin(
                        y=i,
                        name="with imv",
                        box_visible=True,
                        meanline_visible=True))

fig.update_layout(
    autosize=False,
    width=600,
    height=500,
    yaxis=dict(
        title_text="ms",)
    )

fig.show()
